In [1]:
 from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ROOT_DIR="/content/drive/MyDrive/Shakespeare AI"

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
path_to_file= ROOT_DIR+"/shakespeare.txt"

In [ ]:
text=(open(path_to_file,'r').read())

In [ ]:
text[:100]

"\n                     1\n  From fairest creatures we desire increase,\n  That thereby beauty's rose mi"

In [ ]:
print(text[:100])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose mi


In [ ]:
vocab = sorted(set(text))
len(vocab)

84

In [ ]:
char_to_ind={char:ind for ind,char in enumerate(vocab)}
char_to_ind['H']

33

In [ ]:
index_to_char=np.array(vocab)
index_to_char[33]

'H'

In [ ]:
encodded_text=np.array([char_to_ind[c] for c in text])
encodded_text.shape

(5445609,)

In [ ]:
seq_len=120
total_num_seq=len(text)//(seq_len+1)
total_num_seq

45005

In [ ]:
char_dataset=tf.data.Dataset.from_tensor_slices(encodded_text)
type(char_dataset)

tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset

In [ ]:
sequences= char_dataset.batch(seq_len+1,drop_remainder=True)

In [ ]:
def create_seq_targets(seq):
  input_text=seq[:-1]
  target_text=seq[1:]
  return input_text,target_text

In [ ]:
datasets=sequences.map(create_seq_targets)

In [ ]:
for input_text, target_text in datasets.take(1):
  print(input_text.numpy())
  print(''.join(index_to_char[input_text.numpy()]))
  print('\n')
  print(target_text.numpy())
  print(''.join(index_to_char[target_text.numpy()]))

[ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0
  1  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74
  1 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45
 63 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74
 60  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75]

                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But


[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0  1
  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74  1
 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45 63
 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74 60
  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75  1]
                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But 


In [ ]:
batch_size=128
buffer_size=10000
datasets=datasets.shuffle(buffer_size).batch(batch_size,drop_remainder=True)
datasets

<_BatchDataset element_spec=(TensorSpec(shape=(128, 120), dtype=tf.int64, name=None), TensorSpec(shape=(128, 120), dtype=tf.int64, name=None))>

In [ ]:
vocab_size=len(vocab)
vocab_size

84

In [ ]:
embed_dim=64
rnn_neurons=1026

In [ ]:
def sparse_cat_loss(y_true,y_pred):
  return tf.keras.losses.sparse_categorical_crossentropy(y_true,y_pred,from_logits=True)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,GRU,Dense

In [ ]:
def create_model(vocab_size,embed_dim,rnn_neurons,batch_size):
  model=Sequential()
  model.add(Embedding(vocab_size,embed_dim, input_length=None))
  model.add(GRU(rnn_neurons,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
  model.add(Dense(vocab_size))
  model.build(input_shape=(batch_size, None))
  model.compile('adam',loss=sparse_cat_loss)
  return model

# When creating the model, provide the sequence length
#seq_length = 128  # Replace with the actual sequence length of your input data
model = create_model(vocab_size=vocab_size, embed_dim=embed_dim, rnn_neurons=rnn_neurons, batch_size=batch_size)

In [ ]:
model = create_model(vocab_size=vocab_size, embed_dim=embed_dim, rnn_neurons=rnn_neurons, batch_size=batch_size)
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (128, None, 64)             │           5,376 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ (128, None, 1026)           │       3,361,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (128, None, 84)             │          86,268 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,452,820 (13.17 MB)

 Trainable params: 3,452,820 (13.17 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
for input_example_batch, target_example_batch in datasets.take(1):
    example_batch_predictions = model(input_example_batch)

In [ ]:
example_batch_predictions.shape

TensorShape([128, 120, 84])

In [ ]:
sample_indices= tf.random.categorical(example_batch_predictions[0],num_samples=1)
sample_indices=tf.squeeze(sample_indices,axis=-1).numpy()
sample_indices

array([19, 70, 74, 67,  4, 36, 59, 66,  7, 33, 19, 66, 27, 67, 41, 60, 42,
       14, 16, 43, 30, 27, 46,  6,  9, 19, 56, 47, 24, 31, 14,  7, 80, 31,
       22, 62, 62, 56, 32, 50, 28, 28,  4, 74, 44, 20, 79, 52,  3, 41, 70,
        1, 67, 66, 42, 64, 48, 66, 45, 37, 15, 37, 34, 28,  1, 49, 43, 64,
       45, 43,  8, 56, 29, 10, 34, 83, 67, 76, 82, 76, 80,  6, 82, 58, 80,
        7, 58, 76, 60, 29, 61, 70, 43, 21, 75, 65, 58,  3, 59, 57, 37, 29,
       35, 69, 37, 34, 60, 69, 16, 33, 13, 62, 77, 44, 21,  0, 13,  1, 83,
       26])

In [ ]:
index_to_char[sample_indices]

array(['8', 'o', 's', 'l', '&', 'K', 'd', 'k', ')', 'H', '8', 'k', 'B',
       'l', 'P', 'e', 'Q', '3', '5', 'R', 'E', 'B', 'U', '(', '-', '8',
       'a', 'V', '>', 'F', '3', ')', 'y', 'F', ';', 'g', 'g', 'a', 'G',
       'Y', 'C', 'C', '&', 's', 'S', '9', 'x', '[', '"', 'P', 'o', ' ',
       'l', 'k', 'Q', 'i', 'W', 'k', 'T', 'L', '4', 'L', 'I', 'C', ' ',
       'X', 'R', 'i', 'T', 'R', ',', 'a', 'D', '.', 'I', '}', 'l', 'u',
       '|', 'u', 'y', '(', '|', 'c', 'y', ')', 'c', 'u', 'e', 'D', 'f',
       'o', 'R', ':', 't', 'j', 'c', '"', 'd', 'b', 'L', 'D', 'J', 'n',
       'L', 'I', 'e', 'n', '5', 'H', '2', 'g', 'v', 'S', ':', '\n', '2',
       ' ', '}', 'A'], dtype='<U1')

In [ ]:
temp=list(datasets.as_numpy_iterator())

In [ ]:
epochs = 10
model.fit(datasets, epochs=epochs)

Epoch 1/10
351/351 ━━━━━━━━━━━━━━━━━━━━ 55s 141ms/step - loss: 2.9258
Epoch 2/10
351/351 ━━━━━━━━━━━━━━━━━━━━ 82s 145ms/step - loss: 1.6671
Epoch 3/10
351/351 ━━━━━━━━━━━━━━━━━━━━ 51s 140ms/step - loss: 1.4141
Epoch 4/10
351/351 ━━━━━━━━━━━━━━━━━━━━ 53s 142ms/step - loss: 1.3139
Epoch 5/10
351/351 ━━━━━━━━━━━━━━━━━━━━ 52s 141ms/step - loss: 1.2582
Epoch 6/10
351/351 ━━━━━━━━━━━━━━━━━━━━ 83s 145ms/step - loss: 1.2253
Epoch 7/10
351/351 ━━━━━━━━━━━━━━━━━━━━ 81s 144ms/step - loss: 1.1961
Epoch 8/10
351/351 ━━━━━━━━━━━━━━━━━━━━ 82s 145ms/step - loss: 1.1747
Epoch 9/10
351/351 ━━━━━━━━━━━━━━━━━━━━ 82s 144ms/step - loss: 1.1556
Epoch 10/10
351/351 ━━━━━━━━━━━━━━━━━━━━ 51s 140ms/step - loss: 1.1408


In [ ]:
model.save('ShakespeareAI.keras')

In [ ]:
model= create_model(vocab_size,embed_dim,rnn_neurons,batch_size=1)
model.load_weights("ShakespeareAI.keras")


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
model.summary ()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ (1, None, 64)               │           5,376 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_2 (GRU)                          │ (1, None, 1026)             │       3,361,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (1, None, 84)               │          86,268 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,452,820 (13.17 MB)

 Trainable params: 3,452,820 (13.17 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
def generate_text(model, start_seed,gen_size=500, temp=1.0):
  num_generate = gen_size
  input_eval = [char_to_ind[s] for s in start_seed]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []
  temperature = temp

  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(index_to_char[predicted_id])
  return start_seed + "".join(text_generated)

In [ ]:
print(generate_text(model,"DEBJANI",gen_size=1000))

DEBJANIBUTRIUCHISPREGED  MARIANTISTAPTOXNET  FLARIUSIDERCH. Leave Troilus, my lord, might begin she kneel
    Tonguisu'd away his amorousing to the form
    That never gracious init my household
    That lack'd by madness by the women from the Count,
    Will not know us.
  SECOND MURDERER. Heavens, seatelance! why need not find etave.
  SPEED. End will tell,
    His quality string hath done but Romeo
  A Stonia told
  Of serves sorrow over runs away.
  These grows have been too leek, that hast a scratch,
  For thou hast stranger then have traitor'd consjured
  And their conference
    Speaks few to you, for her eye weld I think  
    For, by steen's pardon, God's paid,
    That I have said impute.
  VALENTINE. How mean't it?
  CLARENCE. He struck it, we from Lache Clifford, I'll confise the
    celestial. He same was  
    she amize.
  OPERCON. Princes it, Camillo, and to the rest, and come.
  HERMIA. Now shall I send her too; and to our parts  
    Till all thy stabs whose foot hath 